# 10-11 семинар: Шешімдер ағаштары және Кездейсоқ орман тәжірибеде

**Семинар мақсаты:** Классификация және регрессия сияқты екі түрлі есепте ағаш тәрізді модельдермен жұмыс істеудің толық циклін көрсету. Визуализацияға, интерпретацияға және жеке ағашты ансамбльмен салыстыруға баса назар аудару.

## 1-бөлім: Классификация есебі

**Деректер жиынтығы:** `penguins_size.csv` — пингвиндер туралы деректер. Қарапайым және таза, визуализацияға өте ыңғайлы.
**Мақсаты:** Пингвиннің түрін (`species`) болжауды үйрену.

In [ ]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

df = pd.read_csv('https://raw.githubusercontent.com/yuliya-sabirova/ml-course/main/data/penguins_size.csv')
# Қарапайымдылық үшін бос жолдарды жоямыз
df = df.dropna()
df.info()
df.head()

### 1.1. Деректерді талдау және дайындау

**Түсініктеме:** *Деректерге көз жүгіртейік. `pairplot` белгілердің кластар бойынша бір-бірімен қалай байланысатынын жақсы көрсетеді. Кейбір белгілер жұбы кластарды іс жүзінде мінсіз бөлетінін көруге болады, бұл ағаш тәрізді модельдер үшін жақсы белгі.*

In [ ]:
sns.pairplot(df, hue='species')
plt.show()

**Түсініктеме:** *Ағаштарға сандық деректер қажет. Категориялық белгілерді (`island`, `sex`) `pd.get_dummies` көмегімен сандарға айналдырамыз. Содан кейін деректерді оқыту және тесттік таңдамаларға бөлеміз. **Масштабтау (`StandardScaler`) қажет емес** екеніне назар аударыңыз, себебі ағаштар шектік мәндермен жұмыс істейді және масштабқа сезімтал емес.*

In [ ]:
from sklearn.model_selection import train_test_split

X = pd.get_dummies(df.drop('species', axis=1), drop_first=True)
y = df['species']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=101)

### 1.2. 1-модель: Жеке Шешімдер Ағашы

**Түсініктеме:** *`DecisionTreeClassifier`-ді "қораптан шыққандай" не істей алатынын көру үшін әдепкі параметрлермен оқытайық.*

In [ ]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import classification_report, confusion_matrix

dt_model = DecisionTreeClassifier(random_state=101)
dt_model.fit(X_train, y_train)

dt_preds = dt_model.predict(X_test)

print("--- Decision Tree нәтижелері ---")
print(classification_report(y_test, dt_preds))

**Түсініктеме:** *Енді ең қызығы — ағашты визуализациялаймыз. Ол өте үлкен және күрделі болып шықты. Бұл **артық оқытудың** айқын демонстрациясы: модель оқыту таңдамасындағы әрбір объектіні мінсіз жіктеу үшін өте ерекше ережелер құрды. Жаңа деректерде мұндай күрделі модель нашар жұмыс істеуі мүмкін.*

In [ ]:
from sklearn.tree import plot_tree

plt.figure(figsize=(12, 8))
plot_tree(dt_model, feature_names=X.columns, filled=True)
plt.title("Артық оқытылған шешімдер ағашы")
plt.show()

### 1.3. 2-модель: Кездейсоқ Орман

**Түсініктеме:** *Енді `RandomForestClassifier`-ді қолданамыз. Бұл көптеген осындай ағаштардан тұратын ансамбль. Нәтиже тұрақтырақ және дәлірек болады деп күтеміз.*

In [ ]:
from sklearn.ensemble import RandomForestClassifier

rf_model = RandomForestClassifier(n_estimators=100, random_state=101)
rf_model.fit(X_train, y_train)

rf_preds = rf_model.predict(X_test)

print("--- Random Forest нәтижелері ---")
print(classification_report(y_test, rf_preds))

**Түсініктеме:** *Кездейсоқ орман **белгілердің маңыздылығын** бағалауға мүмкіндік береді — әрбір белгі қорытынды болжамға қандай үлес қосады. Бұл ансамбльдің барлық ағаштары бойынша орташаланған баға.*

In [ ]:
importances = pd.DataFrame(index=X.columns, data=rf_model.feature_importances_, columns=['Importance'])
importances = importances.sort_values('Importance', ascending=False)
print(importances)

**1-бөлім бойынша қорытынды:** *Кездейсоқ орман жеке ағашқа қарағанда жоғары дәлдік көрсетті. Ол сондай-ақ бізге белгілердің маңыздылығы туралы сенімдірек баға берді, тұмсықтың ұзындығы мен биіктігі пингвин түрінің ең маңызды предикторлары екенін көрсетті.*


## 2-бөлім: Регрессия есебі

**Деректер жиынтығы:** `cos(x)` функциясы негізінде шу қосылған синтетикалық деректер.
**Мақсаты:** Ағаш пен орманның тегіс функцияны қалай аппроксимациялайтынын көру.

In [ ]:
# Деректерді генерациялау
x_range = np.linspace(0, 10, 100)
noise = np.random.randn(len(x_range)) * 0.2
y_reg = np.cos(x_range) + noise

X_reg = x_range.reshape(-1, 1)

plt.figure(figsize=(10, 6))
plt.scatter(X_reg, y_reg, label='Шулы деректер')
plt.plot(x_range, np.cos(x_range), 'r--', label='cos(x) нақты функциясы')
plt.legend()
plt.title("Регрессия есебіне арналған синтетикалық деректер")
plt.show()

### 2.1. 1-модель: Регрессия Ағашы

**Түсініктеме:** *`DecisionTreeRegressor`-ді оның болжамдары көрнекі болуы үшін шектелген тереңдікпен (`max_depth=3`) оқытайық.*

In [ ]:
from sklearn.tree import DecisionTreeRegressor

dt_reg = DecisionTreeRegressor(max_depth=3)
dt_reg.fit(X_reg, y_reg)

dt_reg_preds = dt_reg.predict(X_reg)

**Түсініктеме:** *Нәтижені визуализациялаймыз. Ағаштың болжамы — бұл **сатылы функция**. Әрбір "қадам" ағаштың жапырағындағы орташа мәнге сәйкес келеді.*

In [ ]:
plt.figure(figsize=(10, 6))
plt.scatter(X_reg, y_reg, alpha=0.5, label='Деректер')
plt.plot(x_range, dt_reg_preds, 'g-', label='Decision Tree болжамдары (max_depth=3)')
plt.legend()
plt.title("Регрессия ағашымен аппроксимациялау")
plt.show()

### 2.2. 2-модель: Регрессияға арналған Кездейсоқ Орман

**Түсініктеме:** *Енді `RandomForestRegressor`-ді оқытайық. Оның болжамы жүздеген осындай сатылы функциялардың орташа мәні болады.*

In [ ]:
from sklearn.ensemble import RandomForestRegressor

rf_reg = RandomForestRegressor(n_estimators=100)
rf_reg.fit(X_reg, y_reg)

rf_reg_preds = rf_reg.predict(X_reg)

**Түсініктеме:** *Ағаш пен орманның болжамдарын бір графикте салыстырайық. Орманның болжамы әлдеқайда тегіс және шуды ескермей, нақты функцияға жақсырақ ереді.*

In [ ]:
plt.figure(figsize=(12, 8))
plt.scatter(X_reg, y_reg, alpha=0.3, label='Деректер')
plt.plot(x_range, dt_reg_preds, 'g-', label='Decision Tree (max_depth=3)', linewidth=2)
plt.plot(x_range, rf_reg_preds, 'm-', label='Random Forest', linewidth=2)
plt.plot(x_range, np.cos(x_range), 'r--', label='cos(x) нақты функциясы')
plt.legend()
plt.title("Регрессиядағы Ағаш пен Кездейсоқ Орманды салыстыру")
plt.show()

**Семинардың қорытындысы:** *Біз ағаш тәрізді модельдерді классификация үшін де, регрессия үшін де қолдануға болатынына көз жеткіздік. Екі жағдайда да Кездейсоқ Орман түріндегі ансамбльдік тәсіл жеке шешімдер ағашына қарағанда сапалырақ және тұрақты нәтижелер көрсетеді, артық оқытумен тиімді күреседі.*